In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_classes
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from model_saver import model_saver, model_saver_wandb
import training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


c:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\LSTM


In [2]:
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
full_data = pd.read_feather('gru_inputs_new_simple_v6_test.fth')

In [3]:
full_data.columns

Index(['dogid', 'dog_name', 'raceid', 'race_grade', 'date', 'race_time',
       'race_num', 'trackOHE', 'track_name', 'state', 'track_code', 'dist',
       'StartPrice', 'bfSP', 'runtime', 'prev_race', 'prev_race_date',
       'prev_race_track', 'prev_race_state', 'place', 'margin', 'box', 'stats',
       'stats_cols'],
      dtype='object')

In [6]:
full_data.columns


Index(['index', 'dogid', 'dog_name', 'raceid', 'race_grade', 'date',
       'race_time', 'race_num', 'trackOHE', 'track_name', 'state',
       'track_code', 'dist', 'StartPrice', 'bfSP', 'runtime', 'prev_race',
       'prev_race_date', 'prev_race_track', 'prev_race_state', 'place',
       'margin', 'box', 'stats', 'stats_cols'],
      dtype='object')

In [5]:
full_data['stats'].iloc[0]

array([ 3. ,  0. ,  1. ,  0. , 31.3,  6. , -1. , -1. , -1. , -1. , -1. ,
       -1. , -1. , -1. ,  0. , -1. , -1. , -1. , -1. , -1. ])

In [8]:
full_data.bfSP

0                 NaN
1           50.000000
2           18.096412
3          110.000000
4                 NaN
              ...    
1377294           NaN
1377295           NaN
1377296           NaN
1377297           NaN
1377298           NaN
Name: bfSP, Length: 1377299, dtype: float64

In [ ]:
full_data

In [51]:
price_grouped = full_data.groupby('date').sum('price')
price_grouped

,index,dist,bfSP,runtime,place,margin,box
date,,,,,,,
2019-12-01,358281,342213.0,16484.669663,20146.78,3483.0,5377.06,3778
2019-12-02,1391104,414106.0,23516.831798,24447.75,4304.0,6679.18,4557
2019-12-03,2405018,423316.0,21344.926580,25006.41,4180.0,6688.94,4511
2019-12-04,3231843,388231.0,22362.996044,22933.88,3936.0,5935.49,4295
2019-12-05,4606507,462019.0,22374.341779,27089.34,4440.0,6697.12,4713
...,...,...,...,...,...,...,...
2023-09-15,1674274558,500980.0,38827.987758,29401.90,5119.0,7531.62,5465
2023-09-16,1136715635,373294.0,26491.686369,21880.65,3466.0,5494.00,3697
2023-09-17,1492375185,426043.0,36567.861065,24915.99,4565.0,6464.94,4868


In [4]:
full_data.date.min()

datetime.date(2019, 12, 1)

In [5]:
full_data.date.max()

datetime.date(2023, 9, 19)

In [6]:
date_series = pd.date_range(full_data.date.min(),full_data.date.max())

In [18]:
pd.Series(date_series)

0      2019-12-01
1      2019-12-02
2      2019-12-03
3      2019-12-04
4      2019-12-05
          ...    
1384   2023-09-15
1385   2023-09-16
1386   2023-09-17
1387   2023-09-18
1388   2023-09-19
Length: 1389, dtype: datetime64[ns]

In [41]:
dates_in_data = full_data.date.value_counts(sort=False)

In [38]:
dates_in_data = pd.to_datetime(dates_in_data['index'])

In [39]:
dates_in_data

0      2023-07-06
1      2023-04-06
2      2023-06-29
3      2023-07-13
4      2021-07-01
          ...    
1380   2023-04-07
1381   2022-04-15
1382   2023-09-19
1383   2021-04-02
1384   2020-04-10
Name: index, Length: 1385, dtype: datetime64[ns]

In [40]:
[x for x in date_series if x not in dates_in_data]

[Timestamp('2019-12-01 00:00:00', freq='D'),
 Timestamp('2019-12-02 00:00:00', freq='D'),
 Timestamp('2019-12-03 00:00:00', freq='D'),
 Timestamp('2019-12-04 00:00:00', freq='D'),
 Timestamp('2019-12-05 00:00:00', freq='D'),
 Timestamp('2019-12-06 00:00:00', freq='D'),
 Timestamp('2019-12-07 00:00:00', freq='D'),
 Timestamp('2019-12-08 00:00:00', freq='D'),
 Timestamp('2019-12-09 00:00:00', freq='D'),
 Timestamp('2019-12-10 00:00:00', freq='D'),
 Timestamp('2019-12-11 00:00:00', freq='D'),
 Timestamp('2019-12-12 00:00:00', freq='D'),
 Timestamp('2019-12-13 00:00:00', freq='D'),
 Timestamp('2019-12-14 00:00:00', freq='D'),
 Timestamp('2019-12-15 00:00:00', freq='D'),
 Timestamp('2019-12-16 00:00:00', freq='D'),
 Timestamp('2019-12-17 00:00:00', freq='D'),
 Timestamp('2019-12-18 00:00:00', freq='D'),
 Timestamp('2019-12-19 00:00:00', freq='D'),
 Timestamp('2019-12-20 00:00:00', freq='D'),
 Timestamp('2019-12-21 00:00:00', freq='D'),
 Timestamp('2019-12-22 00:00:00', freq='D'),
 Timestamp

In [10]:
merged = pd.merge(date_series,dates_in_data)

TypeError: Can only merge Series or DataFrame objects, a <class 'pandas.core.indexes.datetimes.DatetimeIndex'> was passed

In [3]:
x = torch.rand(10,5)
x

tensor([[0.6093, 0.0979, 0.2050, 0.0206, 0.2658],
        [0.1304, 0.6652, 0.6059, 0.2495, 0.3553],
        [0.6097, 0.1040, 0.6026, 0.6245, 0.4972],
        [0.5594, 0.0055, 0.6893, 0.9701, 0.6273],
        [0.9912, 0.5202, 0.3729, 0.3006, 0.7823],
        [0.1901, 0.1941, 0.4577, 0.2834, 0.6938],
        [0.5569, 0.8647, 0.6258, 0.4970, 0.1522],
        [0.1149, 0.7049, 0.4102, 0.7690, 0.0917],
        [0.9626, 0.5456, 0.0035, 0.8270, 0.4009],
        [0.5970, 0.9059, 0.1292, 0.8892, 0.3155]])

In [6]:
F.softmax(x.mean(dim=1))


C:\Users\Nick\AppData\Local\Temp\ipykernel_25352\2877334043.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(x.mean(dim=1))


tensor([0.0788, 0.0926, 0.1009, 0.1096, 0.1122, 0.0892, 0.1063, 0.0941, 0.1072,
        0.1093])

In [9]:
model_name = 'rosy-night-60'
all_price_df = pd.read_feather(f'C:/Users/Nick/Documents/GitHub/grvmodel/Python/DATA/model_all_price/{model_name} - all_price_df.fth')
all_price_df.to_excel(f'{model_name} - all_price_df.xlsx')

In [182]:
race_profit_df = all_price_df[['flat_races','profit_relu<30','track']].groupby(['track','flat_races']).sum().reset_index()
race_profit_df['profit'] = np.where(race_profit_df['profit_relu<30']>0, 1, 0)
race_profit_df['loss'] = np.where(race_profit_df['profit_relu<30']<0, 1, 0)

In [184]:
race_profit_df = race_profit_df.groupby('track').sum()
race_profit_df['win_rate'] = race_profit_df['profit']/(race_profit_df['profit']+race_profit_df['loss'])

C:\Users\Nick\AppData\Local\Temp\ipykernel_11352\3910978394.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [187]:
race_profit_df.sort_values(['profit_relu<30'])

,profit_relu<30,profit,loss,win_rate
track,,,,
Dapto,-3131.717664,88,337,0.207059
Nowra,-1092.371980,108,280,0.278351
Wagga Wagga,-707.244500,109,268,0.289125
Taree,-651.455756,73,274,0.210375
Bathurst,-613.733242,29,104,0.218045
Muswellbrook,-163.760988,5,14,0.263158
Gunnedah,-101.565401,121,359,0.252083
Broke Hill,378.368772,13,51,0.203125
Richmond Straight,545.737862,48,177,0.213333


In [ ]:
flat_date_df = all_price_df[['flat_date','outlay < 30','profit < 30','bet amount','profit','profit_relu','profit_relu<30']].groupby('flat_date').sum().cumsum().reset_index()
flat_date_df

In [ ]:
testing = all_price_df[['flat_date','track','outlay < 30','profit < 30','bet amount','profit','profit_relu','profit_relu<30']].groupby(['flat_date','track']).sum().reset_index()
testing = testing.set_index('flat_date')
testing

In [ ]:
df = testing
df['dummy'] = df['track'].astype('category',copy=False).cat.codes
df = df.reset_index()
df

In [ ]:
df.pivot(index='flat_date',columns='track',values='profit_relu<30').rolling(300,min_periods=0).sum().dropna().melt(ignore_index=False).reset_index()

In [ ]:
df.pivot(index='flat_date',columns='track',values='profit_relu<30').rolling(300,min_periods=0).sum().dropna().melt(ignore_index=False).reset_index()

In [ ]:
testg = all_price_df[['flat_date','track','outlay < 30','profit < 30','bet amount','profit','profit_relu','profit_relu<30']].groupby(['flat_date','track']).sum()
testg

In [ ]:
testug = all_price_df[['flat_date','track','outlay < 30','profit < 30','bet amount','profit','profit_relu','profit_relu<30']].groupby(['flat_date','track']).sum().reset_index().groupby('track')

In [ ]:
for i,j in testug:
    print(i,j)
    print(j['profit_relu<30'].cumsum())

In [ ]:
all_price_df[['flat_date','outlay < 30','profit < 30','bet amount','profit']]

In [ ]:
flat_date_df = all_price_df[['flat_date','outlay < 30','profit < 30','bet amount','profit']].groupby('flat_date').sum().cumsum().reset_index()
flat_date_df

In [ ]:
flat_date_df['date'] = all_price_df['flat_date'].unique()

In [ ]:
flat_date_df

In [ ]:
all_price_df[['flat_date','outlay < 30','profit < 30','bet amount','profit']].groupby('flat_date',as_index=False).cumsum(axis=1)

In [ ]:
model_name = 'zesty-aardvark-17'
all_price_df = pd.read_feather(f'C:/Users/Nick/Documents/GitHub/grvmodel/Python/DATA/model_all_price/{model_name} - all_price_df.fth')
all_price_df.to_excel(f'{model_name} - all_price_df.xlsx')

In [ ]:
daily_profit = all_price_df.groupby(['track','flat_date']).sum()

In [ ]:
(all_price_df['bet_relu']/all_price_df['bet amount']).mean()

In [ ]:
all_price_df['bet_relu'].max()

In [ ]:
all_price_df['bet amount'].max()

In [ ]:
all_price_df['month'] = all_price_df.flat_date.apply(lambda x: x.month)

In [ ]:
all_price_df['month']

In [ ]:
grouped = all_price_df.groupby('month', as_index=False).sum()

In [ ]:
all_price_df.columns

In [ ]:
all_price_df.groupby(['flat_races'])['pred_prob'].max()

In [ ]:
stat_values = pd.read_feather(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\expected_stat_values.fth')
stat_values.columns

In [ ]:
stat_values

In [3]:
stat_values2 = pd.read_feather(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\all_stat_values.fth')
stat_values2.columns
# stat_values2.mean()
# stat_values2.drop(columns=['Track','dateF'], inplace=True)
# stat_values2.isna().sum()
# stat_values2.sample(frac=0.5).to_excel('all stat values.xlsx')

Index(['index', 'Track', 'dateF', 'dist_x', 'box', 'inside', 'midfield',
       'wide', 'weight', 'DogGrade', 'dist_last__1', 'box_last__1',
       'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1',
       'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1',
       'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1',
       'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1',
       'wins_last_1', 'weight_', 'min_time_', 'min_split_time_',
       'min_split_time_v1', 'last_start_price', 'last_start_prob'],
      dtype='object')

In [193]:
stat_values2.dateF = pd.to_datetime(stat_values2.dateF)
stat_values2['date'] = stat_values2.dateF.apply(lambda x: x.strftime('%Y-%m'))

In [198]:
stat_values2.groupby('Track').mean().to_excel('mean_states_by_track.xlsx')

C:\Users\Nick\AppData\Local\Temp\ipykernel_11352\2649673154.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [195]:
stat_values2[stat_values2['Track']=='Dapto'].groupby('date').mean()

C:\Users\Nick\AppData\Local\Temp\ipykernel_11352\1595245060.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,dist_x,box,inside,midfield,wide,weight,DogGrade,dist_last__1,box_last__1,speed_avg_1,split_speed_avg_1,split_margin_avg_1,margin_avg_1,run_home_speed_1,races_1,wins_1,wins_last_1,weight_,last_start_price
date,,,,,,,,,,,,,,,,,,,
2019-12,452.964539,4.397163,0.276596,0.471631,0.251773,30.210638,5.666667,279.159574,2.478723,10.478677,9.376283,-0.344149,3.282518,9.493107,1.265957,-0.148936,-0.241135,18.698582,9.106028
2020-01,413.611801,4.338509,0.298137,0.437888,0.263975,30.036957,5.881988,383.549689,3.959627,15.634787,14.719932,-0.004286,5.518634,14.713065,3.838509,0.503106,0.021739,27.574845,15.260776
2020-02,407.333333,4.427984,0.288066,0.436214,0.275720,29.897942,5.448560,397.691358,4.230453,16.413343,14.267880,0.012551,5.403786,14.324669,7.419753,1.082305,0.086420,28.694650,15.786420
2020-03,436.259259,4.440741,0.288889,0.448148,0.262963,29.464444,5.444444,403.511111,4.285185,16.197829,11.211567,-0.203037,6.118556,11.249267,9.062963,1.166667,0.103704,28.123333,14.854630
2020-04,470.095491,4.469496,0.265252,0.474801,0.259947,29.649072,5.188329,447.880637,4.228117,16.694363,13.188925,-0.100371,6.221167,13.067399,10.167109,1.843501,0.119363,28.886207,15.754960
2020-05,487.235119,4.494048,0.252976,0.494048,0.252976,29.698214,5.375000,460.705357,4.446429,16.946884,14.290464,0.087411,6.408780,14.405659,12.455357,2.086310,0.125000,29.374107,18.283155
2020-06,446.800752,4.443609,0.293233,0.428571,0.278195,29.213910,5.515038,420.800752,4.304511,16.495188,14.198367,-0.017180,5.970677,14.409315,14.199248,1.936090,0.097744,28.214286,19.162519
2020-07,429.605479,4.421918,0.273973,0.468493,0.257534,29.569315,5.602740,421.328767,4.265753,16.809107,13.830045,0.019562,6.096164,13.954972,15.194521,2.287671,0.126027,29.118904,17.217123
2020-08,417.720238,4.500000,0.279762,0.434524,0.285714,28.985119,5.815476,403.690476,4.464286,16.502719,14.440411,-0.020119,6.468452,14.326643,16.982143,2.488095,0.089286,28.055952,18.270119


In [ ]:
stat_values2.mean()

In [ ]:
stat_values2 = pd.read_feather(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\all_stat_values_normed.fth')
stat_values2.columns
stat_values2.max()
# stat_values2.drop(columns=['Track','dateF'], inplace=True)
stat_values2.isna().sum()

In [ ]:
stat_values3 = stat_values2.sample(frac=0.1)

In [ ]:
plot = stat_values3.boxplot()

In [ ]:
plot.plot()

In [ ]:
stat_values2 = pd.read_feather(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\all_stat_values_normed.fth')
stat_values2.columns

In [ ]:
stat_values2.mean()

In [ ]:
stat_values2 = stat_values2[stat_values2['Track']=='Christchurch']

In [ ]:
date_groups = stat_values2.groupby(['Track','dateF']).mean()-stat_values2.groupby(['Track']).mean().mean()

In [ ]:
stat_values2.groupby(['Track','dateF']).mean().mean()

In [ ]:
date_groups

In [ ]:
date_groups.reset_index().to_excel('date_mean_values_mean.xlsx')

In [ ]:
standard_dev = stat_values2.groupby(['Track','dist_x'], as_index=False).std()

In [ ]:
stat_values = pd.read_feather(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\DATA\ft_races_ALL.fth')

In [4]:
gru_in  = pd.read_feather(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\DATA\gru_inputs_simple_v8_kitchen_sink_vic.fth')

In [5]:
gru_in.state.value_counts()

VIC    375450
Name: state, dtype: int64

In [5]:
gru_in.columns

Index(['dogid', 'dog_name', 'raceid', 'race_grade', 'date', 'race_time',
       'race_num', 'trackOHE', 'track_name', 'state', 'track_code', 'dist',
       'bfSP', 'runtime', 'prev_race', 'prev_race_date', 'prev_race_track',
       'prev_race_state', 'place', 'margin', 'box', 'stats', 'stats_cols'],
      dtype='object')

In [10]:
date_counts = gru_in.date.value_counts(sort=False)

In [29]:
betfair_df = gru_in[['state','track_name','bfSP', 'place']]
betfair_df.place.value_counts()

1.0    174980
2.0    174482
3.0    173972
4.0    173291
5.0    170527
6.0    159721
7.0    130513
8.0     89233
Name: place, dtype: int64

In [18]:
encoder_layer  = nn.TransformerEncoderLayer(d_model=28,nhead=4)

In [20]:
src = torch.rand(10, 32, 28)
out = encoder_layer(src)

In [21]:
out

tensor([[[ 1.7869, -0.8647,  0.5560,  ...,  0.2022, -1.9550,  1.2832],
         [ 0.0889, -2.2476,  0.5172,  ..., -0.1187,  0.6325, -1.4083],
         [-0.9231,  0.6058, -1.9773,  ..., -0.3424, -0.7776, -0.4887],
         ...,
         [ 1.0046, -0.3500, -2.3323,  ...,  0.6823, -0.0217,  0.0732],
         [ 0.2600,  0.6188, -0.2993,  ..., -0.7176, -0.3630,  0.3129],
         [ 0.7341, -1.0566, -0.4283,  ...,  0.9994,  1.0963,  0.7103]],

        [[ 1.3126,  1.0803,  0.2953,  ...,  0.9589, -0.7199, -0.8945],
         [ 1.9534,  0.9476, -0.2368,  ..., -1.5488, -1.4649,  0.6498],
         [-0.4237, -0.9453, -0.1558,  ...,  1.3038, -0.5297,  0.8432],
         ...,
         [ 1.0003, -1.2751,  0.2437,  ...,  1.1744,  0.2357,  0.7831],
         [-0.7525,  1.0474, -1.4354,  ...,  1.2866, -0.1033,  1.4060],
         [ 1.8023, -1.4204, -1.5545,  ...,  1.3084, -0.6450,  0.4208]],

        [[-0.5019,  0.9963, -1.5199,  ..., -0.8217, -0.3658,  1.1161],
         [ 0.9208, -0.6710, -1.8714,  ...,  0

In [32]:
race_with_win = betfair_df[betfair_df['place']==1]
race_with_win = race_with_win.dropna()
race_with_win['prob'] = race_with_win.bfSP.apply(lambda x: 1/x)
race_with_win['log_prob'] = np.log(race_with_win.prob)
race_with_win['log_loss'] = -(np.log(race_with_win.prob))
race_with_win.groupby(['state','track_name']).mean()

bfSP  place      prob  log_prob  log_loss
state track_name                                                      
NSW   Bathurst           7.148397    1.0  0.290738 -1.498868  1.498868
      Broke Hill         7.018654    1.0  0.318189 -1.450985  1.450985
      Bulli              6.470834    1.0  0.328065 -1.383777  1.383777
      Casino             7.478608    1.0  0.268672 -1.571568  1.571568
      Dapto              7.106694    1.0  0.283366 -1.513412  1.513412
      Dubbo              7.018573    1.0  0.303080 -1.463084  1.463084
      Gosford            6.829770    1.0  0.287474 -1.514591  1.514591
      Goulburn           6.752432    1.0  0.313412 -1.435319  1.435319
      Grafton            7.645470    1.0  0.285633 -1.533464  1.533464
      Gunnedah           6.673631    1.0  0.289035 -1.497354  1.497354
      Lismore            7.795277    1.0  0.278265 -1.569599  1.569599
      Maitland           6.435517    1.0  0.363870 -1.291757  1.291757
      Muswellbrook       8.004548    1.0  0.201676 -1.815496  1.815496
      Nowra              6.688864    1.0  0.296641 -1.476868  1.476868
      Richmond           7.487607    1.0  0.281657 -1.547197  1.547197
      Richmond Straight  6.162980    1.0  0.349037 -1.322953  1.322953
      Taree              8.423349    1.0  0.248250 -1.665247  1.665247
      Temora             7.604434    1.0  0.281278 -1.560955  1.560955
      The Gardens        7.478931    1.0  0.276779 -1.559210  1.559210
      Wagga Wagga        7.794213    1.0  0.252937 -1.634686  1.634686
      Wauchope           7.909894    1.0  0.271721 -1.572879  1.572879
      Wentworth Park     7.409974    1.0  0.288304 -1.517275  1.517275
NT    Darwin             6.558557    1.0  0.292175 -1.480243  1.480243
NZ    Auckland           7.763983    1.0  0.260780 -1.624434  1.624434
      Christchurch       7.644273    1.0  0.267085 -1.605599  1.605599
      Otago              8.009584    1.0  0.219717 -1.764706  1.764706
      Palmerston North   6.753710    1.0  0.285128 -1.521621  1.521621
      Southland          7.853355    1.0  0.263534 -1.625694  1.625694
      Waikato            6.939528    1.0  0.283919 -1.539163  1.539163
      Wanganui           7.692012    1.0  0.251321 -1.657400  1.657400
QLD   Albion Park        7.420033    1.0  0.276366 -1.547006  1.547006
      Bundaberg          7.116739    1.0  0.280708 -1.533306  1.533306
      Capalaba           6.343265    1.0  0.332794 -1.367790  1.367790
      Ipswich            7.845275    1.0  0.276404 -1.563819  1.563819
      Rockhampton        6.528290    1.0  0.286659 -1.493636  1.493636
      Townsville         7.578045    1.0  0.260517 -1.610516  1.610516
SA    Angle Park         6.455395    1.0  0.305523 -1.446061  1.446061
      Gawler             6.121035    1.0  0.310395 -1.416709  1.416709
      Mount Gambier      6.102445    1.0  0.306556 -1.418247  1.418247
      Murray Bridge      5.969950    1.0  0.344467 -1.319920  1.319920
TAS   Devonport          8.476713    1.0  0.303527 -1.493840  1.493840
      Hobart             7.618808    1.0  0.300831 -1.507741  1.507741
      Launceston         7.289467    1.0  0.280146 -1.544786  1.544786
VIC   Ballarat           7.057728    1.0  0.276223 -1.541242  1.541242
      Bendigo            7.367546    1.0  0.274257 -1.560842  1.560842
      Cranbourne         7.489521    1.0  0.258244 -1.610480  1.610480
      Geelong            7.070691    1.0  0.283231 -1.524107  1.524107
      Healesville        6.862196    1.0  0.314048 -1.426237  1.426237
      Horsham            6.594394    1.0  0.311309 -1.426206  1.426206
      Sale               7.033049    1.0  0.292377 -1.500903  1.500903
      Sandown Park       6.963776    1.0  0.278384 -1.534523  1.534523
      Shepparton         7.175495    1.0  0.272447 -1.558490  1.558490
      The Meadows        7.276407    1.0  0.278244 -1.540755  1.540755
      Traralgon          6.846186    1.0  0.279615 -1.530366  1.530366
      Warragul           7.298953    1.0  0.281426 -1

In [ ]:

def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [ ]:
track_combos = gru_in[['state','track_name','prev_race_track']][gru_in['track_name']!=gru_in['prev_race_track']].value_counts(sort=False).reset_index().sort_values(by=['track_name',0], ascending=False)

In [3]:
import networkx as nx
import pandas as pd
import plotly.graph_objects as go

In [204]:
track_combos = gru_in[['state','track_name','prev_race_track']][gru_in['prev_race_state']!='-1']
track_groups = track_combos.groupby('track_name',sort=False)
links = track_groups.value_counts(normalize=True).reset_index()
links[links['track_name']=='Sandown Park'].head(8).prev_race_track.tolist()

['Sandown Park',
 'The Meadows',
 'Warragul',
 'Geelong',
 'Sale',
 'Ballarat',
 'Bendigo',
 'Cranbourne']

In [136]:
track_combos = gru_in[['state','track_name','prev_race_track']][gru_in['track_name']!=gru_in['prev_race_track']][gru_in['prev_race_track']!='-1']
track_groups = track_combos.groupby('track_name',sort=False)
links = track_groups.value_counts(normalize=True).reset_index()
links.columns  = ['track_name', 'state', 'prev_race_track','weight']
links = links[links['weight']>0.01]

C:\Users\Nick\AppData\Local\Temp\ipykernel_11352\2733357941.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [93]:
links = links[links['state']!='NZ']
links

,track_name,state,prev_race_track,weight
0,Albion Park,QLD,Ipswich,0.550092
1,Albion Park,QLD,Capalaba,0.245908
2,Albion Park,QLD,Bundaberg,0.046067
3,Albion Park,QLD,Casino,0.030029
4,Albion Park,QLD,Lismore,0.025013
...,...,...,...,...
2685,Young,NSW,Richmond,0.038677
2686,Young,NSW,Potts Park,0.015596
2687,Young,NSW,Dapto,0.014348
2688,Young,NSW,Lithgow,0.013724


In [137]:
track_names = gru_in[['track_name','state']].value_counts().reset_index().to_excel('tracks2.xlsx')

In [138]:
track_coords = pd.read_excel('tracks.xlsx')

In [139]:
track_coords = track_coords[['track_name', 'latitude', 'long']]

In [140]:
links = links.merge(track_coords, on='track_name')
links

,track_name,state,prev_race_track,weight,latitude,long
0,Albion Park,QLD,Ipswich,0.550092,-34.566667,150.8
1,Albion Park,QLD,Capalaba,0.245908,-34.566667,150.8
2,Albion Park,QLD,Bundaberg,0.046067,-34.566667,150.8
3,Albion Park,QLD,Casino,0.030029,-34.566667,150.8
4,Albion Park,QLD,Lismore,0.025013,-34.566667,150.8
...,...,...,...,...,...,...
660,Young,NSW,Richmond,0.038677,-34.300000,148.3
661,Young,NSW,Potts Park,0.015596,-34.300000,148.3
662,Young,NSW,Dapto,0.014348,-34.300000,148.3
663,Young,NSW,Lithgow,0.013724,-34.300000,148.3


In [141]:
links['coords'] = links.apply(lambda x: (x['long'],x['latitude']), axis=1)
links_dict = links[['track_name','coords','long','latitude']].set_index('track_name').to_dict()

In [142]:
links_dict['coords']

{'Albion Park': (150.8, -34.566667),
 'Angle Park': (138.5575223, -34.8595915),
 'Auckland': (174.762782, -36.847317),
 'Ballarat': (143.85497, -37.5609002),
 'Bathurst': (149.5834743, -33.4192653),
 'Bendigo': (144.284004, -36.7593416),
 'Broke Hill': (141.4653416, -31.9558883),
 'Bulli': (150.9132453, -34.3344113),
 'Bundaberg': (152.3506879, -24.8643995),
 'Cannington': (115.9330031, -32.0198746),
 'Capalaba': (153.1927, -27.522994),
 'Casino': (153.05, -28.866667),
 'Christchurch': (172.638, -43.5308),
 'Coonabarabran': (149.266667, -31.25),
 'Coonamble': (148.4, -30.95),
 'Cranbourne': (145.279, -38.105),
 'Dapto': (150.7947394, -34.4933663),
 'Darwin': (130.833333, -12.45),
 'Devonport': (146.333333, -41.166667),
 'Dubbo': (148.601111, -32.256944),
 'Gawler': (138.745, -34.59806),
 'Geelong': (144.35, -38.15),
 'Gosford': (151.341667, -33.426667),
 'Goulburn': (149.618611, -34.754722),
 'Grafton': (152.933333, -29.683333),
 'Gunnedah': (150.25, -30.96667),
 'Healesville': (145.48

In [156]:
G = nx.from_pandas_edgelist(links, source='prev_race_track', target='track_name',
                            edge_attr='weight',)

In [159]:
nx.set_node_attributes(G, links_dict['coords'], 'pos')
nx.set_node_attributes(G, links_dict['long'], 'long')
nx.set_node_attributes(G, links_dict['latitude'], 'lat')

In [160]:
G.edges

EdgeView([('Ipswich', 'Albion Park'), ('Ipswich', 'Bundaberg'), ('Ipswich', 'Capalaba'), ('Ipswich', 'Casino'), ('Ipswich', 'Coonabarabran'), ('Ipswich', 'Darwin'), ('Ipswich', 'Grafton'), ('Ipswich', 'Rockhampton'), ('Ipswich', 'Lismore'), ('Ipswich', 'Kempsey'), ('Ipswich', 'Moree'), ('Ipswich', 'Taree'), ('Ipswich', 'Townsville'), ('Ipswich', 'Wauchope'), ('Albion Park', 'Capalaba'), ('Albion Park', 'Bundaberg'), ('Albion Park', 'Casino'), ('Albion Park', 'Lismore'), ('Albion Park', 'Rockhampton'), ('Albion Park', 'Grafton'), ('Albion Park', 'Darwin'), ('Albion Park', 'Moree'), ('Albion Park', 'Townsville'), ('Albion Park', 'Wauchope'), ('Albion Park', 'Wentworth Park'), ('Capalaba', 'Bundaberg'), ('Capalaba', 'Casino'), ('Capalaba', 'Lismore'), ('Capalaba', 'Grafton'), ('Capalaba', 'Rockhampton'), ('Capalaba', 'Darwin'), ('Capalaba', 'Moree'), ('Capalaba', 'Townsville'), ('Capalaba', 'Wauchope'), ('Bundaberg', 'Rockhampton'), ('Bundaberg', 'Townsville'), ('Bundaberg', 'Darwin'), ('

In [161]:
edge_x = []
edge_y = []
widths = np.array([w for *_, w in G.edges.data('weight')])
for edge in G.edges():
    print(edge)
    print(G.nodes[edge[0]])
    print(G.nodes[edge[1]])
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)
    # weight.extend([edge['weight'],edge[1]['weight'],None])

('Ipswich', 'Albion Park')
{'pos': (152.766667, -27.616667), 'long': 152.766667, 'lat': -27.616667}
{'pos': (150.8, -34.566667), 'long': 150.8, 'lat': -34.566667}
('Ipswich', 'Bundaberg')
{'pos': (152.766667, -27.616667), 'long': 152.766667, 'lat': -27.616667}
{'pos': (152.3506879, -24.8643995), 'long': 152.3506879, 'lat': -24.8643995}
('Ipswich', 'Capalaba')
{'pos': (152.766667, -27.616667), 'long': 152.766667, 'lat': -27.616667}
{'pos': (153.1927, -27.522994), 'long': 153.1927, 'lat': -27.522994}
('Ipswich', 'Casino')
{'pos': (152.766667, -27.616667), 'long': 152.766667, 'lat': -27.616667}
{'pos': (153.05, -28.866667), 'long': 153.05, 'lat': -28.866667}
('Ipswich', 'Coonabarabran')
{'pos': (152.766667, -27.616667), 'long': 152.766667, 'lat': -27.616667}
{'pos': (149.266667, -31.25), 'long': 149.266667, 'lat': -31.25}
('Ipswich', 'Darwin')
{'pos': (152.766667, -27.616667), 'long': 152.766667, 'lat': -27.616667}
{'pos': (130.833333, -12.45), 'long': 130.833333, 'lat': -12.45}
('Ipswich

In [165]:
len(widths)*3

1239

In [170]:
edge_trace = go.Scattergeo(
    lon=edge_x, lat=edge_y,
    line=dict(width=widths, color='#888'),
    hoverinfo='none',
    mode='lines')

ValueError: 
    Invalid value of type 'numpy.ndarray' received for the 'width' property of scattergeo.line
        Received value: array([0.6091989 , 0.0545233 , 0.21786476, 0.02238806, 0.02083333,
       0.08389262, 0.01705757, 0.11868818, 0.03812317, 0.01015228,
       0.0685112 , 0.01054018, 0.12725546, 0.02097187, 0.47967586,
       0.29169649, 0.07024793, 0.10210611, 0.18636127, 0.08135145,
       0.07718121, 0.04611331, 0.15479582, 0.01636829, 0.01111005,
       0.02914652, 0.03557312, 0.02879232, 0.02416006, 0.04633004,
       0.02348993, 0.01054018, 0.03608737, 0.01074169, 0.3925039 ,
       0.04653371, 0.02684564, 0.38863722, 0.49106905, 0.02305665,
       0.0629156 , 0.02335025, 0.04479578, 0.26472941, 0.02030457,
       0.03120205, 0.01139601, 0.24786325, 0.02731457, 0.06609574,
       0.0390463 , 0.06240409, 0.01124903, 0.01113628, 0.05279188,
       0.01054018, 0.01899335, 0.56610395, 0.01451062, 0.01677852,
       0.36263503, 0.1025405 , 0.47055709, 0.13825479, 0.1918032

In [166]:
node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scattergeo(
    lon=node_x, lat=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=widths))

In [167]:
for node, adjacencies in enumerate(G.adjacency()):
    print(node,adjacencies[0])

0 Ipswich
1 Albion Park
2 Capalaba
3 Bundaberg
4 Casino
5 Lismore
6 Rockhampton
7 Grafton
8 Gawler
9 Angle Park
10 Murray Bridge
11 Mount Gambier
12 Waikato
13 Auckland
14 Palmerston North
15 Wanganui
16 Christchurch
17 Geelong
18 Ballarat
19 Bendigo
20 Warrnambool
21 Shepparton
22 The Meadows
23 Healesville
24 Horsham
25 Warragul
26 Sandown Park
27 Cranbourne
28 Sale
29 Traralgon
30 Dubbo
31 Bathurst
32 Richmond
33 Goulburn
34 Wentworth Park
35 Temora
36 Lithgow
37 Wagga Wagga
38 Gunnedah
39 Young
40 Bulli
41 Coonamble
42 Dapto
43 Richmond Straight
44 Gosford
45 Broke Hill
46 Nowra
47 Maitland
48 The Gardens
49 Townsville
50 Mandurah
51 Cannington
52 Northam
53 Taree
54 Wauchope
55 Southland
56 Otago
57 Coonabarabran
58 Tamworth
59 Kempsey
60 Muswellbrook
61 Moree
62 Darwin
63 Launceston
64 Devonport
65 Hobart
66 Potts Park
67 Taranaki
68 Tokoroa


In [168]:
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append(adjacencies[0]+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

In [169]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=True, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=True, zeroline=False, showticklabels=False))
                )
fig.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
widths = np.array([w for *_, w in G.edges.data('weight')])

pos = nx.spring_layout(G, seed=7, k=0.25)  # positions for all nodes - seed for reproducibi
plt.figure(figure(figsize=(10,10)))
# nodes
nx.draw_networkx_nodes(G, pos, node_size=100)

# edges
nx.draw_networkx_edges(G, pos, width=widths*100, edge_color='grey')  # using a 10x scale factor here

# labels
nx.draw_networkx_labels(G, pos, font_size=12, font_family="sans-serif", font_color='white')

ax = plt.gca()
ax.margins(0.08)
plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
pos

In [ ]:
track_combos['weight'] = track_groups.value_counts()

In [ ]:
stat_values.columns

In [ ]:
stat_values.PIR.head(50)

In [ ]:
stat_values['first_out_avg_1'].max()

In [4]:
prev_full = pd.read_feather(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\results-df-merged-prices_cut.fth")

In [7]:
jet = prev_full[prev_full['DogName']=="SCREAMING JET"]
jet

,DogId,DogName,RaceId,RaceGrade,dateF,dateF1,date,RaceTime,RaceNum,Track,...,race_time,tracksOnehot,PIR,PIR_adj,run_home_speed_v1,run_home_speed,split_speed_v1,RunHomeTime,StartProb,start_pos
1033918,710478414,SCREAMING JET,833066712,Maiden Heat,2022-10-25,NaT,25 Oct 22,03:12PM,2,Bulli,...,2023-06-14 15:12:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",4433,4433,11.043084,17.795372,1498.333222,16.42,0.181818,4.0
1041022,710478414,SCREAMING JET,838005788,Maiden Final,2022-11-01,NaT,01 Nov 22,08:17PM,5,Bulli,...,2023-06-14 20:17:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",2658,2658,10.858417,17.372177,749.166611,16.82,0.105263,2.0
1144636,710478414,SCREAMING JET,870678499,Maiden,2023-02-10,NaT,10 Feb 23,12:08PM,3,Goulburn,...,2023-06-14 12:08:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,1,NaN,NaN,NaN,NaN,0.571429,1.0
1149371,710478414,SCREAMING JET,870434713,Grade 5 Heat,2023-02-15,NaT,15 Feb 23,10:05PM,10,Wentworth Park,...,2023-06-14 22:05:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",322,322,14.158155,17.335926,1919.265227,24.46,0.208333,3.0
1159522,710478414,SCREAMING JET,874633874,Grade 5 Final,2023-02-25,NaT,25 Feb 23,06:09PM,1,Wentworth Park,...,2023-06-14 18:09:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",651,651,14.186575,17.385680,1066.258459,24.39,0.200000,6.0
1163736,710478414,SCREAMING JET,875176171,Grade 5,2023-03-01,NaT,01 Mar 23,09:34PM,8,Wentworth Park,...,2023-06-14 21:34:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",832,832,14.153429,17.364322,1919.265227,24.42,0.500000,8.0
1171103,710478414,SCREAMING JET,877730717,Grade 5,2023-03-08,NaT,08 Mar 23,09:48PM,9,Wentworth Park,...,2023-06-14 21:48:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",321,321,14.172351,17.371435,2399.081534,24.41,0.645161,3.0
1177557,710478414,SCREAMING JET,877658265,Grade 5,2023-03-15,NaT,15 Mar 23,07:07PM,1,Wentworth Park,...,2023-06-14 19:07:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",866,866,14.017743,17.167479,799.693845,24.70,0.555556,8.0
1185140,710478414,SCREAMING JET,881692603,Grade 5,2023-03-22,NaT,22 Mar 23,09:49PM,9,Wentworth Park,...,2023-06-14 21:49:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",432,432,14.177089,17.286455,738.178933,24.53,0.625000,4.0
1228569,710478414,SCREAMING JET,894754643,Grade 5,2023-05-03,NaT,03 May 23,09:08PM,7,Wentworth Park,...,2023-06-14 21:08:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",543,543,14.036304,17.181391,959.632613,24.68,0.384615,5.0


In [ ]:
prev_full.columns

In [ ]:
prev_full['Track']

In [ ]:
races = prev_full[['DogId', 'RaceId']]

In [ ]:
dog_races = races.groupby('DogId')
race_dogs = races.groupby('RaceId')

In [ ]:
race_dogs.get_group('870678784')

In [ ]:
races.head(8)

In [ ]:
race_dogs.get_group('622583706')

In [ ]:
first_races = dog_races.get_group('622583706')
races_to_check = list(first_races.RaceId)
print(races_to_check)
set_dogs = set('622583706')
checked_races = set()

while races_to_check:
    race = races_to_check.pop(0)
    checked_races.add(race)
    # print(race)
    dogs = race_dogs.get_group(race).DogId.tolist()
    # print(dogs)
    for dog in dogs:
        if dog not in set_dogs:
            dog_race_set = set(dog_races.get_group(dog).RaceId)
            races_to_add = dog_race_set-checked_races
            races_to_check.extend(races_to_add)
            set_dogs.add(dog)
            print(f"{len(set_dogs)=},{len(races_to_check)=}")


In [ ]:
len(set_dogs)

In [ ]:
len(set(races.DogId))

In [ ]:
no_splits = prev_full[(prev_full['SplitTimes'].isna())&(prev_full['Place']>1)]

In [ ]:
prev_full_tail = prev_full.tail(10_000)

In [ ]:
prev_full_tail.to_excel('prev full 10K.xlsx')

In [ ]:
prev_full.groupby('State').count()

In [ ]:
no_splits.groupby(['State']).count()

In [ ]:
nosplit_group = no_splits.groupby(['State','Track']).count()

In [ ]:
stat_cols= ['box','inside','midfield','wide','weight','DogGrade','dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price']

In [ ]:
og_df = pd.read_feather(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\gru_inputs_new_simple_test.fth')
og_stats = pd.DataFrame(og_df.stats.to_list(),columns=stat_cols)
og_stats['date'] = og_df.date
test_og = og_stats.mean()
og_stats.max()

In [ ]:
og_df = pd.read_feather(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\gru_inputs_new_simple_test.fth')
date = datetime.date(2022,8,1)
og_df = og_df[og_df['date']<date]
og_stats = pd.DataFrame(og_df.stats.to_list(),columns=stat_cols)
og_stats['date'] = og_df.date
test_og = og_stats.mean()
og_stats.mean()

In [ ]:
og_stats_daily = og_stats.groupby('date').mean()

In [ ]:
og_df = pd.read_feather(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\Database Updater\DATA\gru_inputs_simple_TESTING.fth')
date = datetime.date(2022,8,1)
og_df = og_df[og_df['date']<date]
og_stats = pd.DataFrame(og_df.stats.to_list(),columns=stat_cols)
og_stats['date'] = og_df.date
test_og2 = og_stats.mean()
og_stats.mean()

In [ ]:
test_og-test_og2

In [ ]:
og_df = pd.read_feather(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\gru_inputs_simple_OG.fth')
og_stats = pd.DataFrame(og_df.stats.to_list(),columns=stat_cols)
og_stats.mean()


In [ ]:
og_df = pd.read_feather(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\gru_inputs_new_simple_testing_OG_v2.fth')
og_stats = pd.DataFrame(og_df.stats.to_list(),columns=stat_cols)
og_stats.mean()
